<a href="https://colab.research.google.com/github/benspanswick/QTW/blob/master/Homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [2]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [4]:
case_8 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/case_8.csv')

In [5]:
case_8.shape

(114321, 133)

In [6]:
case_8.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,...,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,...,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,G,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,1.630686,7.464411,C,4.145098,9.191265,2.436402,2.483921,2.301630,9.031859,1.312910,15.447413,6.507647,3.798396,11.636386,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.763110,GUV,1.093088e+00,C,3.056144,1.876031,2.743454,5.093328,8.206416,C,A,1.622151,2.161633,3.615077,8.122387,14.579479,0.741471,0,...,0.582367,5.475185,3.852883,0.665758,6.457952,7.622554,8.303967,1.250721,12.091623,6.866414,2.890289,5.296716,2.642828,1.505335,11.791360,B,1.825361,4.247858,A,3.365314,U,G,10.308044,10.548051,2.291218,10.595357,8.364651,3.168970e+00,1.291218,2.737596,6.822439,3.549938,0.598896,AF,1.672658,3.239542,1.957825,0,1.925763,1.739389
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,C,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,...,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,G,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,...,0.763925,5.498902,3.423944,0.832518,7.375480,6.746988,6.942002,1.334611,18.256352,8.507281,2.503055,4.872157,2.573664,0.113967,12.554274,B,2.230754,1.990131,B,2.643678,J,G,13.777666,10.574713,1.511063,4.949609,7.180722,5.655086e-01,1.166281,1.956521,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,1.630686,7.464411,C,4.145098,8.742359,2.436402,2.483921,1.496569,9.031859,1.050328,15.447413,6.320087,3.798396,10.991098,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.414567,HIT,1.093088e+00,E,1.698129,1.876031,2.743454,5.093328,8.206416,C,A,1.622151,2.161633,6.083151,8.122387,13.375598,0.741471,0,...,0.582367,5.475185,3.852883,0.665758,6.457952,7.622554,7.667624,1.250721,12.091623,6.866414,2.890289,5.296716,2.642828,1.081045,11.791360,C,2.152620,4.181284,A,3.365314,T,G,14.097099,10.548051,2.291218,8.303857,8.364651,3.168970e+00,1.291218,2.737596,6.822439,3.549938,0.919812,Z,1.672658,3.239542,2.030373,0,1.925763,1.739389


In [7]:
dataset = case_8
del dataset['ID']
del dataset['v22']
dataset = dataset.reset_index(drop=True)

In [8]:
dataset = pd.get_dummies(dataset)

In [9]:
dataset.head()

,target,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v23,v25,v26,v27,v28,v29,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,...,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z
0,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,-1.716131e-08,0.139412,1.720818,3.393503,0.590122,8.880867,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1.630686,7.464411,4.145098,9.191265,2.436402,2.483921,2.301630,9.031859,1.312910,15.447413,6.507647,3.798396,11.636386,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.763110,1.093088e+00,3.056144,1.876031,2.743454,5.093328,8.206416,1.622151,2.161633,3.615077,8.122387,14.579479,0.741471,0,1.237184,14.305766,7.182551,12.924965,2.216597,10.795169,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,-2.785053e-07,0.113997,2.244897,5.306122,0.836005,7.499999,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0.797415,8.304757,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,-4.805344e-07,0.148843,1.308269,2.303640,8.926662,8.874521,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1.630686,7.464411,4.145098,8.742359,2.436402,2.483921,1.496569,9.031859,1.050328,15.447413,6.320087,3.798396,10.991098,2.080911,4.923222,3.832270,0.841045,0.222300,17.773592,6.414567,1.093088e+00,1.698129,1.876031,2.743454,5.093328,8.206416,1.622151,2.161633,6.083151,8.122387,13.375598,0.741471,0,1.237184,10.138920,7.182551,12.924965,2.216597,10.795169,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [10]:
#print(dataset.head())
#print(dataset.describe())
print(dataset.shape)
#for col in dataset.columns: 
#    print(col) 
dataset.columns[dataset.isnull().any()]

(114321, 477)


Index([], dtype='object')

In [11]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [12]:
X, y = dataset.iloc[:,:-1],dataset.iloc[:,-1]

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.33, random_state=42)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [43]:
rfc=RandomForestRegressor(random_state=42)

In [49]:
param_grid = { 
    #'n_estimators': [200, 500],
    #'max_features': ['auto', 'sqrt', 'log2'],
    #'max_depth' : [4,5,6,7,8],
    #'criterion' :['gini', 'entropy']
}

In [50]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs

In [41]:
rfc1 = RandomForestClassifier(**CV_rfc.best_params_)

In [42]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
pred=rfc1.predict(X_test)

In [52]:
from sklearn.metrics import log_loss
log_loss(y_test,pred)

0.06591723213774231

In [28]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.9981445157186025


In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV


cv = 2


paramGrid = {#'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               #'max_features': ['auto', 'sqrt'],
               #'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               #'min_samples_split': [2, 5, 10],
               #'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

fit_params={"eval_set" : [[X_test, y_test]]}


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores

gridsearch = GridSearchCV(rf, paramGrid, verbose=1,             
         cv=TimeSeriesSplit(n_splits=cv).get_n_splits([X_test, y_test]))

gridsearch.fit(X_train, y_train)



Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: ignored

In [18]:
gridsearch.best_params_

{'bootstrap': True}

In [21]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [22]:
best_grid = gridsearch.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

Model Performance
Average Error: 0.0000 degrees.
Accuracy = 100.00%.


In [16]:
gridsearch.best_params_

{'bootstrap': True}

In [ ]:
import xgboost.sklearn as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

cv = 2

paramGrid = {#"subsample" : [0.5, 0.8],
             #'max_depth': [3, 4, 5, 6, 7, 8, 9],
             #'colsample_bytree': [0.5, 0.45],
             #'min_child_weight': [1, 2, 3],
             #'learning_rate': [0.01, 0.07],
             'n_estimators': [150, 500]
             }


fit_params={"early_stopping_rounds":10, 
            "eval_metric" : "logloss", 
            "eval_set" : [[X_test, y_test]]}

model = xgb.XGBRegressor(objective='reg:squarederror')

gridsearch = GridSearchCV(model, paramGrid, verbose=1,             
         cv=TimeSeriesSplit(n_splits=cv).get_n_splits([X_test, y_test]))

gridsearch.fit(X_train, y_train, **fit_params)

Fitting 2 folds for each of 336 candidates, totalling 672 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Streaming output truncated to the last 5000 lines.
[23]	validation_0-logloss:0.505142
[24]	validation_0-logloss:0.498591
[25]	validation_0-logloss:0.492214
[26]	validation_0-logloss:0.486263
[27]	validation_0-logloss:0.48021
[28]	validation_0-logloss:0.474252
[29]	validation_0-logloss:0.468201
[30]	validation_0-logloss:0.46261
[31]	validation_0-logloss:0.456949
[32]	validation_0-logloss:0.451579
[33]	validation_0-logloss:0.446274
[34]	validation_0-logloss:0.440663
[35]	validation_0-logloss:0.435362
[36]	validation_0-logloss:0.42992
[37]	validation_0-logloss:0.424754
[38]	validation_0-logloss:0.41982
[39]	validation_0-logloss:0.414727
[40]	validation_0-logloss:0.409959
[41]	validation_0-logloss:0.40499
[42]	validation_0-logloss:0.40038
[43]	validation_0-logloss:0.395527
[44]	validation_0-logloss:0.391063
[45]	validation_0-logloss:0.386638
[46]	validation_0-logloss:0.382286
[47]	validation_0-logloss:0.377721
[48]	validation_0-logloss:0.373493
[49]	validation_0-logloss:0.369074
[50]	valid

In [ ]:

clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist,  n_iter = 25, scoring = 'f1', error_score = 0, verbose = 3, n_jobs = -1)

numFolds = 5
folds = KFold(n_splits = numFolds, shuffle = True)

estimators = []
results = np.zeros(len(X))
score = 0.0
for train_index, test_index in folds.split(X):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()
    clf.fit(X_train, y_train, fit_params = {"early_stopping_rounds":5}  )
    estimators.append(clf.best_estimator_)
    results[test_index] = clf.predict(X_test)
    score += f1_score(y_test, results[test_index])
score /= numFolds

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )


In [ ]:

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


In [ ]:
y_test = random_search.predict_proba(test)
results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
results_df.to_csv('submission-random-grid-search-xgb-porto-01.csv', index=False)